In [73]:
!pip install skimage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached skimage-0.0.tar.gz (757 bytes)
ERROR: Could not find a version that satisfies the requirement skimage (from versions: 0.0)
ERROR: No matching distribution found for skimage


In [74]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Tasks: 
### - Image classification: dogs vs cats

### - Method: Linear classification

### - Features: histogram

### - Color: grayscale

In [75]:
import numpy as np
import pandas as pd
import os
import cv2 


dir_monhoc = '/content/drive/MyDrive/CS231.N12.KHCL/Baitap/Dataset/dogscats_small'
dir_test_images = '/content/drive/MyDrive/CS231.N12.KHCL/Baitap/Dataset/dogscats_small/test'
dir_train_images = '/content/drive/MyDrive/CS231.N12.KHCL/Baitap/Dataset/dogscats_small/train'

### Calculate Histogram

In [76]:
def TinhHist(pathfilename):
  img = cv2.imread(pathfilename,0)
  hist = cv2.calcHist([img],[0],None,[256],[0,256])
  size = img.shape[0] * img.shape[1]
  hist = hist / size
  return hist

### Calculate HOG

In [77]:
from skimage import feature

def TinhHOG(pathfilename):
    img = cv2.imread(pathfilename,0)
    img =cv2.resize(img, (64, 128))
    (hog, hog_image) = feature.hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=True, transform_sqrt=True)
    return hog

In [78]:
def ExtractFeature(pathfilename,option = 0):
  if(option==0):
    return TinhHist(pathfilename)
  else:
    return TinhHOG(pathfilename)



###  load_data

In [79]:
def ReadData(path_to_files, option):
  labels = []
  features = []
  for file_name in os.listdir(path_to_files):
    label = file_name.split(os.path.sep)[-1].split(".")[0]
    labels.append(label)
    feature = ExtractFeature(os.path.join(path_to_files, file_name), option)
    features.append(feature)
  features = np.array(features)
  features = np.reshape(features,(features.shape[0],-1))
  labels = np.array(labels)
  return features, labels
    

In [80]:
from sklearn.linear_model import LogisticRegression

x_train,y_train = ReadData(dir_train_images,0)
x_test,y_test = ReadData(dir_test_images,0)

In [81]:
model = LogisticRegression(random_state = 0)
model.fit(x_train,y_train)
model.score(x_test, y_test)


0.45

### Apply KNN and Linear classification to HOG features

In [82]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

x_train,y_train = ReadData(dir_train_images,1)
x_test,y_test = ReadData(dir_test_images,1)

### HOG KNN

In [83]:
n_neighbors = [1, 3, 5, 7, 9, 11, 13]
for k in range(1, 21):
  model = KNeighborsClassifier(n_neighbors=k)
  model.fit(x_train,y_train)
  acc = model.score(x_test,y_test)
  print(f"Độ chính xác của mô hình với k = {k}: {acc}")

Độ chính xác của mô hình với k = 1: 0.55
Độ chính xác của mô hình với k = 2: 0.7
Độ chính xác của mô hình với k = 3: 0.65
Độ chính xác của mô hình với k = 4: 0.7
Độ chính xác của mô hình với k = 5: 0.65
Độ chính xác của mô hình với k = 6: 0.65
Độ chính xác của mô hình với k = 7: 0.6
Độ chính xác của mô hình với k = 8: 0.65
Độ chính xác của mô hình với k = 9: 0.65
Độ chính xác của mô hình với k = 10: 0.65
Độ chính xác của mô hình với k = 11: 0.65
Độ chính xác của mô hình với k = 12: 0.65
Độ chính xác của mô hình với k = 13: 0.5
Độ chính xác của mô hình với k = 14: 0.65
Độ chính xác của mô hình với k = 15: 0.45
Độ chính xác của mô hình với k = 16: 0.5
Độ chính xác của mô hình với k = 17: 0.55
Độ chính xác của mô hình với k = 18: 0.55
Độ chính xác của mô hình với k = 19: 0.55
Độ chính xác của mô hình với k = 20: 0.65


### HOG Linear Classification

In [84]:
model = LogisticRegression(random_state = 0)
model.fit(x_train, y_train)
model.score(x_test, y_test)

0.75